In [70]:
import os
from dotenv import load_dotenv
from operator import itemgetter
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.docstore.document import Document

from langchain_community.vectorstores import Neo4jVector

from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

from langchain.chains.question_answering import load_qa_chain


In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
embedding_model = OpenAIEmbeddings()

# NEO4J connection data
load_dotenv()
password = os.getenv("NEO4J_PASSWORD")
url = os.getenv("NEO4J_URL")
username = os.getenv("NEO4J_USER_NAME")

dYKvEn-UNbtRGWCyFzory8MUEbjc-fw-nQ53tqF5_3sneo4j+ssc://ba68e959.databases.neo4j.ioNone


In [79]:
vectorstore = Neo4jVector.from_existing_index(
        embedding_model,
        url=url,
        username=username,
        password=password,
        index_name="Chunk",
        )
retriever = vectorstore.as_retriever()


In [211]:
template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory_1 = ConversationBufferMemory(memory_key="chat_history", input_key="human_input",  chat_memory=retrieved_chat_history)

chain_1 = load_qa_chain(
    llm, chain_type="stuff", memory=memory_1, prompt=prompt
)

In [214]:
chat_history = chain_1.memory.chat_memory.messages
chat_history

[HumanMessage(content='What is my name? '),
 AIMessage(content=" I'm sorry, I don't know your name. Can you tell me your name? If you have provided me with your name before, I can remember and use it in future conversations. However, if you want to keep your name a secret, I completely respect that decision."),
 HumanMessage(content='My name is Finn '),
 AIMessage(content=' Nice to meet you, Finn! I will remember your name for our future conversations. Is there anything else you would like to talk about?'),
 HumanMessage(content='What is my name? '),
 AIMessage(content=' Your name is Finn. Is there anything else you would like to talk about?')]

In [207]:
ingest_to_db = messages_to_dict(chat_history)
ingest_to_db

[{'type': 'human',
  'data': {'content': 'What is my name? ',
   'additional_kwargs': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': " I'm sorry, I don't know your name. Can you tell me your name? If you have provided me with your name before, I can remember and use it in future conversations. However, if you want to keep your name a secret, I completely respect that decision.",
   'additional_kwargs': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'human',
  'data': {'content': 'My name is Finn ',
   'additional_kwargs': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': ' Nice to meet you, Finn! I will remember your name for our future conversations. Is there anything else you would like to talk about?',
   'additional_kwargs': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}},
 {'type':

In [205]:
query = "What is my name? "
chain_1({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' Your name is Finn. Is there anything else you would like to talk about?'}

In [204]:
query = "My name is Finn "
chain_1({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' Nice to meet you, Finn! I will remember your name for our future conversations. Is there anything else you would like to talk about?'}

In [215]:
retrieved_chat_history = ChatMessageHistory(messages=chat_history)
retrieved_chat_history

ValidationError: 1 validation error for ChatMessageHistory
messages -> 0
  __init__() missing 1 required positional argument: 'content' (type=type_error)

In [217]:
retrieve_from_db = json.loads(json.dumps(ingest_to_db))
retrieve_from_db

[{'type': 'human',
  'data': {'content': 'What is my name? ',
   'additional_kwargs': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': " I'm sorry, I don't know your name. Can you tell me your name? If you have provided me with your name before, I can remember and use it in future conversations. However, if you want to keep your name a secret, I completely respect that decision.",
   'additional_kwargs': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'human',
  'data': {'content': 'My name is Finn ',
   'additional_kwargs': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': ' Nice to meet you, Finn! I will remember your name for our future conversations. Is there anything else you would like to talk about?',
   'additional_kwargs': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}},
 {'type':

In [ ]:
ingest_to_db = messages_to_dict(extracted_messages)
ingest_to_db

In [219]:
retrieved_messages = messages_from_dict([])
retrieved_messages

[]